# 1st Homework - data visualisation and web scraping (deadline October 18th)

  * In this homework you should download (*scrape*) data from the web, process it and visualise.
  * The objective is to download data from server https://www.psp.cz/en/sqw/hlasovani.sqw?o=8 regarding voting in the Chamber of Deputies of the Parliament of the Czech Republic, save it in a tabular form and create visualisations, which make exploration of the data easier and show interesting information about it.

> **Unfortunately, the web page is only in Czech. However, it should be possible to navigate it. Please contact your tutorial teacher in case of any questions.**

## Data

 * Download data from all votings of the current Chamber of Deputies (since 2017). Download details of voting of particular deputies.
 * Data should contain basic information about the voting - number of meeting, number of voting, point of the meeting and date.

## Instructions

> **Homework is assigned in a way that you have space for invention. Thinking of the _exact solution path_ is part of the assignment. Originality will be taken into account in the evaluation.**

**Basic points of the assignment (8 points)**:
  * Write Python script for downloading data. Download the data and save it in a suitable machine-readable format.
  * **Wait at least 1 second between two consecutive requests to the server, to not overload it.**
  * In the second part of the notebook, work with the data loaded from a local file. File(s) with downloaded data should be submitted as well (so the reviewer do not have to download the data again).
  * Create visualisations to show the following:
    * Deputies changing their parliamentary clubs.
    * Attendance of individual deputies in the votings. Attendance of parliamentary clubs in the votings.
    * How often individual parliamentary clubs vote the same and different.
    * Are deputies in the same parliamentary club voting the same? Who are the biggest rebels?

**Further points of assignment**, each for 2 points (maximum for the homework is 12):
  * Visualise some time development in the data (e.g. attendance, change of agreement in voting among individual parliamentary clubs, etc.)
  * Find individual deputies who have the most similar voting or attendance.
  * Try to find particular voting, where deputies voted the most differently than traditionally.
  
## Tips and tricks
  * Import libraries at the beginning of the notebook, or the beginning of scraping and visualisation parts.
  * Use markdown cells (like this one) and headings to make orientation in the notebook easier.
  * Select plots and visualisation matching the information you want to show. You can see galleries of libraries `matplotlib` and `seaborn` for inspiration.

## Submission notes

  * Follow instructions at https://courses.fit.cvut.cz/BIE-VZD/homeworks/index.html
  * Submit **Jupyter Notebook** (possibly with additional scripts) and **file(s)** with downloaded data
  * Reviewer may allow you to finish or correct your homework to achieve additional points. However, the first version is crucial.

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import re
import lxml.html


class Deputy:
    def __init__(self, name, id):
        self.name = name
        self.id = id
        self.generate_info()
        
    def generate_info(self):
        time.sleep(0.5)
        if self.id == '6165':  # speaker exeption. His page is changed
            self.polit_group = 'ANO 2011'
            self.member_from = '26.10.2013'
            return
        self.url = str('https://public.psp.cz/en/sqw/detail.sqw?id=' + str(self.id) + '&o=8&l=cz')
        r = requests.get(self.url)
        soup = BeautifulSoup(r.content.decode('windows-1252', 'ignore'), 'html.parser')
        # Politic group
        polit_groups = [i for i in soup.find_all('a') if 'Political group' in str(i)]
        for idx, p_group in enumerate(polit_groups):
            polit_groups[idx] = str(str(p_group).split('Political group ', 1)[1])[:-4]
        self.polit_group = polit_groups
        # Member from
        self.member_from = str(soup.find_all('li')[0]).split('member  from ')[1].replace('\xa0', '')[:-5]
        
    def load_to_csv(self, path):
        out = str(self.name) + ';' + str(self.id) + ';' + str(self.polit_group) + ';' + str(self.member_from) + '\n'
        with open(path, 'w') as file:
            file.write(out)

            
class Voting():
    def __init__(self, id, meeting, topic, date, result):
        self.id = id
        self.meeting = meeting
        self.topic = topic
        self.date = date
        self.result = result
        
        self.url = 'https://public.psp.cz/en/sqw/hlasy.sqw?g=' + str(self.id) + '&l=en'
#     add list with each deputy vote
        

In [13]:

def download_data(url):
    vote_list = []
    deput_list = []
    # Step one
    r = requests.get(url)
    soup = BeautifulSoup(r.content.decode('windows-1252', 'ignore'), 'html.parser')
    ahrefs = []
    
    for link in soup.find_all('a'):
        ahrefs.append(link.get('href'))
    
    meeting_links = []
    url = 'https://public.psp.cz/en/sqw/'
    
    for elem in ahrefs:
        if elem.startswith('hl.sqw?o=8&s=') and len(elem) <= 15:
            meeting_links.append(url + elem)
    # Step two
    r = requests.get(meeting_links[0])
    r.encoding="windows-1252"
    soup = BeautifulSoup(r.content, 'lxml')

    # soup.find_all('a', href=re.compile("phlasa.sqw"))  -> alternative
    link = soup.select_one("a[href^='phlasa.sqw']") 
    link = re.search(r'"([^"]*)"', str(link))
    link = url + link.group()[1:-1]  # link == 'https://public.psp.cz/en/sqw/phlasa.sqw?o=8&l=en&pg=1'
    
    r = requests.get(link)
    soup = BeautifulSoup(r.content.decode('windows-1252', 'ignore'), 'html.parser')
    ahrefs.clear()
    
    for link in soup.find_all('a'):
        ahrefs.append(link.get('href'))
    
#     for elem in ahrefs:
#         if elem.startswith('hlasy.sqw?g='):
#             id_list.append(str(elem.split('hlasy.sqw?g=')[1])[:-5])

    pg_list = []
    for elem in ahrefs:
        if elem.startswith('phlasa.sqw?o=8&l=en&pg='):
            pg_list.append(int(elem.split('phlasa.sqw?o=8&l=en&pg=')[1]))

    for i in range(max(pg_list) + 1):
        if i == 0:
            continue
        url = 'https://public.psp.cz/en/sqw/phlasa.sqw?o=8&l=en&pg=' + str(i)
        r = requests.get(url)
        soup = BeautifulSoup(r.content.decode('windows-1252', 'ignore'), 'html.parser')
        ahrefs.clear()

        # Step three
        # Creating Votings data
        for tr in soup.find_all('table')[0].find_all('tr'):
            tds = []
            for td in tr.find_all('td'):
        #         display(td)
                tds.append(td)
            if not tds:
                continue
            # Meeting
            meeting = tds.pop(0)
            meeting = int(meeting.text)
            # ID
            id = tds.pop(0)
            id = str(str(id.find('a').get('href')).split('hlasy.sqw?g=', 1)[1])[:-5]
            # Topic
            tds.remove(tds[0])
            topic = tds.pop(0).text
            # Date
            date = tds.pop(0).text.replace('\xa0', '')
            # Result
            result = tds.pop(0).text.replace('\u00E8', '\u010D').replace('\xa0', ' ').replace('\u00F8', '\u0159').replace('\u00EC', '\u00E8')
            if result == 'Přijato':
                result = 'Accepted'
            elif result == 'Zamítnuto':
                result = 'Denied'
            elif result == 'Přijato (zmatečné)':
                result = 'Accepted (invalid)'
            vote_list.append(Voting(id, meeting, topic, date, result))
#         for i in vote_list:
#             display(i.id)
        # Step four
        # Creating Depute data
        for vote in vote_list:
            
            
    
    

    
        
    

In [14]:
# Here we start scraping the data. 
# url with a form
url = 'https://www.psp.cz/en/sqw/hlasovani.sqw?o=8'

download_data(url)

 ...]

In [54]:
r = requests.get(links)
r.encoding="windows-1252"
soup = BeautifulSoup(r.content, 'lxml')
# https://public.psp.cz/en/sqw/hlasy.sqw?g=67018&l=en is a link example

ahrefs.clear()
for link in soup.find_all('a'):
    ahrefs.append(link.get('href'))

In [56]:
votings = []
url = 'https://public.psp.cz/en/sqw/'
for elem in ahrefs:
    if elem.startswith('hlasy.sqw?g='):
        votings.append(url + elem)
votings

['https://public.psp.cz/en/sqw/hlasy.sqw?g=67018&l=en',
 'https://public.psp.cz/en/sqw/hlasy.sqw?g=67019&l=en',
 'https://public.psp.cz/en/sqw/hlasy.sqw?g=67020&l=en',
 'https://public.psp.cz/en/sqw/hlasy.sqw?g=67021&l=en',
 'https://public.psp.cz/en/sqw/hlasy.sqw?g=67022&l=en',
 'https://public.psp.cz/en/sqw/hlasy.sqw?g=67023&l=en',
 'https://public.psp.cz/en/sqw/hlasy.sqw?g=67024&l=en',
 'https://public.psp.cz/en/sqw/hlasy.sqw?g=67025&l=en',
 'https://public.psp.cz/en/sqw/hlasy.sqw?g=67026&l=en',
 'https://public.psp.cz/en/sqw/hlasy.sqw?g=67027&l=en',
 'https://public.psp.cz/en/sqw/hlasy.sqw?g=67028&l=en',
 'https://public.psp.cz/en/sqw/hlasy.sqw?g=67029&l=en',
 'https://public.psp.cz/en/sqw/hlasy.sqw?g=67030&l=en',
 'https://public.psp.cz/en/sqw/hlasy.sqw?g=67031&l=en',
 'https://public.psp.cz/en/sqw/hlasy.sqw?g=67032&l=en',
 'https://public.psp.cz/en/sqw/hlasy.sqw?g=67033&l=en']

In [8]:
deputes = []
r = requests.get(votings[0])
r.encoding="windows-1252"
soup = BeautifulSoup(r.content, 'lxml')
for link in soup.find_all('li'):
    deputes.append(link)

In [9]:
votings[0]

'https://public.psp.cz/en/sqw/hlasy.sqw?g=67018&l=en'

In [10]:
deputes = []
r = requests.get(votings[0])
soup = BeautifulSoup(r.content.decode('windows-1252', 'ignore'), 'html.parser')
uls = soup.find_all('ul', {'class':'results'})
for ul in uls:
    for li in ul.find_all('li'):
        name = li.find('a').text.replace('\u00E8', '\u010D').replace('\xa0', ' ').replace('\u00F8', '\u0159').replace('\u00EC', '\u00E8')
        vote = 
        id = str(li.find('a').get('href').split('id=', 1)[1])[:4]
        deputes.append(Deputy(str(name),str(id)))
deputes

'url:https://public.psp.cz/en/sqw/detail.sqw?id=6254&o=8&l=cz'

[<a href="fsnem.sqw?id=1292&amp;o=8">Political group ANO 2011</a>]

'url:https://public.psp.cz/en/sqw/detail.sqw?id=6150&o=8&l=cz'

[<a href="fsnem.sqw?id=1292&amp;o=8">Political group ANO 2011</a>]

'url:https://public.psp.cz/en/sqw/detail.sqw?id=6428&o=8&l=cz'

...

[<a href="fsnem.sqw?id=1296&amp;o=8">Political group Mayors and Independents</a>]

'url:https://public.psp.cz/en/sqw/detail.sqw?id=6417&o=8&l=cz'

[<a href="fsnem.sqw?id=1296&amp;o=8">Political group Mayors and Independents</a>]

'url:https://public.psp.cz/en/sqw/detail.sqw?id=6524&o=8&l=cz'

[<a href="fsnem.sqw?id=1296&amp;o=8">Political group Mayors and Independents</a>,
 <a href="fsnem.sqw?id=1296&amp;o=8">Political group Mayors and Independents</a>]

In [ ]:
    
# for elem in ahrefs:
#     if elem.startswith('hlasy.sqw?g='):
#         id_list.append(str(elem.split('hlasy.sqw?g=')[1])[:-5])
    
# pg_list = []
# for elem in ahrefs:
#     if elem.startswith('phlasa.sqw?o=8&l=en&pg='):
#         pg_list.append(int(elem.split('phlasa.sqw?o=8&l=en&pg=')[1]))
    
# for i in range(max(pg_list) + 1):
#     if i == 0 or i == 1:
#         continue
#     url = 'https://public.psp.cz/en/sqw/phlasa.sqw?o=8&l=en&pg=' + str(i)
#     r = requests.get(url)
#     soup = BeautifulSoup(r.content.decode('windows-1252', 'ignore'), 'html.parser')
#     ahrefs.clear()
    
#     for link in soup.find_all('a'):
#         ahrefs.append(link.get('href'))
    
#     for elem in ahrefs:
#         if elem.startswith('hlasy.sqw?g='):
#             id_list.append(str(elem.split('hlasy.sqw?g=')[1])[:-5])
    
    